In [ ]:
import torch
import torchvision.models as models
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import pandas as pd
from datetime import datetime


In [ ]:


def load_trained_model(model_path):
    base_model = models.resnet18(weights='IMAGENET1K_V1')
    num_features = base_model.fc.in_features
    base_model.fc = nn.Sequential(
        nn.Linear(num_features, 256),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(256, 13),
        nn.Sigmoid()
    )
    
    device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
    
    checkpoint = torch.load(model_path, map_location=device)
    base_model.load_state_dict(checkpoint['model_state_dict'])
    base_model = base_model.to(device)
    
    return base_model, device

def load_and_transform_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])
    
    try:
        img = Image.open(image_path).convert('RGB')
        return transform(img)
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None

def predict(image_path, model, device):
    img_tensor = load_and_transform_image(image_path)
    if img_tensor is None:
        return None
    
    model.eval()
    with torch.no_grad():
        outputs = model(img_tensor.unsqueeze(0).to(device))
        
    return outputs.cpu().numpy()

def classify_images(image_paths, model_path='best_model.pth', confidence_threshold=0.5):
    # Categories in the same order as training
    categories = ['fruit', 'bread', 'cookware', 'seafood', 'wine', 
                 'meal', 'cheese', 'meat', 'food', 'beverage', 
                 'dairy', 'vegetable', 'dessert']
    
    model, device = load_trained_model(model_path)
    model.eval()
    
    results = []
    for image_path in image_paths:
        try:
            pred = predict(image_path, model, device)
            
            if pred is None:
                raise Exception("Failed to process image")
                
            pred = pred[0] 
            
            # Get categories where prediction exceeds threshold
            detected_categories = [
                (cat, float(score)) 
                for cat, score in zip(categories, pred) 
                if score > confidence_threshold
            ]
            
            # Sort by confidence
            detected_categories.sort(key=lambda x: x[1], reverse=True)
            
            results.append({
                'image_path': image_path,
                'predictions': detected_categories
            })
            
        except Exception as e:
            print(f"Error processing {image_path}: {e}")
            results.append({
                'image_path': image_path,
                'error': str(e)
            })
    
    return results


In [ ]:
data = pd.read_csv('data/paintings_image_path.csv')
images_to_cat = data.copy()
images_paths = images_to_cat['image_path'].tolist()
data

In [ ]:

results = classify_images(images_paths, model_path='models/best_model.pth', confidence_threshold=0.5)
results_df = pd.DataFrame(results)
display(results_df)

final_df = pd.merge(images_to_cat, results_df, on='image_path')
display(final_df)

timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
final_df['timestamp'] = timestamp

final_df.to_csv('data/paintings_with_food_cv.csv', index=False)

for result in results:
    print(f"\nImage: {result['image_path']}")
    if 'error' in result:
        print(f"Error: {result['error']}")
    else:
        print("Detected categories:")
        for category, confidence in result['predictions']:
            print(f"- {category}: {confidence:.2%}")